In [1]:
from huggingface_hub import login
login('')

/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from src.data_utils import SIFT50MDataset
import os
from datasets import load_dataset, Dataset

sift_dataset = load_dataset(
    'amazon-agi/SIFT-50M',
    name='closed_ended_comparison',
    split='train',
    trust_remote_code=True
)
sift_dataset = sift_dataset.shuffle(seed=90)
df = sift_dataset.select(range(1000)).to_pandas()
print(len(sift_dataset))
allowed_values = ["common_voice_de", "multilingual_librispeech_de", "common_voice_en",'vctk_en'] # The datasets were taking too long to get donwloaded so I restricted it to certain subsets

filtered_df = df[df["data_source"].isin(allowed_values)]

# Count number of entries per value
counts = filtered_df["data_source"].value_counts()
print(counts)
sift_data = Dataset.from_pandas(filtered_df)
# Define the base datasets paths (replace with your actual paths)


/home/jovyan/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3613791
data_source
common_voice_en                133
multilingual_librispeech_de     68
common_voice_de                 55
vctk_en                          4
Name: count, dtype: int64


In [53]:
filtered_df.iloc[100].messages

array([{'role': 'user', 'content': array([{'text': 'Which audio clip mentions a character from a series?', 'audio_path': None},
              {'text': None, 'audio_path': '/path/to/common_voice_en_22177961.wav'},
              {'text': None, 'audio_path': '/path/to/common_voice_en_22462206.wav'}],
             dtype=object)}                                                                                                    ,
       {'role': 'assistant', 'content': array([{'text': 'Audio clip 2 mentions a character from a series.', 'audio_path': None}],
             dtype=object)}                                                                                                      ],
      dtype=object)

In [2]:
filtered_df_1 = filtered_df[filtered_df["data_source"].isin(['vctk_en'])]
print(filtered_df_1.iloc[1]['messages'])

[{'role': 'user', 'content': array([{'text': 'Which audio clip has a transcript related to a sports event?', 'audio_path': None},
        {'text': None, 'audio_path': '/path/to/p238_009.wav'},
        {'text': None, 'audio_path': '/path/to/p239_435.wav'}],
       dtype=object)}
 {'role': 'assistant', 'content': array([{'text': 'The second audio clip has a transcript related to a sports event.', 'audio_path': None}],
       dtype=object)}                                                                                                                       ]


In [3]:
sift_data = Dataset.from_pandas(filtered_df_1)


In [2]:
base_datasets_root = "/home/jovyan/.cache/huggingface/datasets"
base_datasets_paths = {
    "common_voice_de": None, # No longer needs a path, handled by load_dataset
    "multilingual_librispeech_de": None, # No longer needs a path, handled by load_dataset
    "common_voice_en": None, # No longer needs a path, handled by load_dataset
    "vctk_en": "./vctk_corpus" # VCTK still needs a root path for torchaudio
}

# Create an instance of the SIFT50MDataset
sift_iterable_dataset = SIFT50MDataset(sift_dataset=sift_data, base_datasets_paths=base_datasets_paths)

# Iterate through the dataset and print the audio path and some metadata
print("Iterating through SIFT-50M dataset:")
for i, data in enumerate(sift_iterable_dataset):
    if i >= 5:  # Print first 5 examples for demonstration
        break
    print(data)
print("Demonstration complete.")

common_voice_de None
mapping csv


100%|██████████| 567993/567993 [00:28<00:00, 19929.04it/s]


Common Voice de mapping saved to ./data/common_voice_de_mapping.csv
multilingual_librispeech_de None
common_voice_en None
mapping csv


100%|██████████| 1070066/1070066 [01:02<00:00, 17107.21it/s]


Common Voice en mapping saved to ./data/common_voice_en_mapping.csv
vctk_en ./vctk_corpus


/home/jovyan/.local/opt/conda/envs/ainmc/lib/python3.11/site-packages/torchaudio/_backend/utils.py:213: UserWarning: In 2.9, this function's implementation will be changed to use torchaudio.load_with_torchcodec` under the hood. Some parameters like ``normalize``, ``format``, ``buffer_size``, and ``backend`` will be ignored. We recommend that you port your code to rely directly on TorchCodec's decoder instead: https://docs.pytorch.org/torchcodec/stable/generated/torchcodec.decoders.AudioDecoder.html#torchcodec.decoders.AudioDecoder.
  warnings.warn(
/home/jovyan/.local/opt/conda/envs/ainmc/lib/python3.11/site-packages/torchaudio/_backend/ffmpeg.py:88: UserWarning: torio.io._streaming_media_decoder.StreamingMediaDecoder has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/

Iterating through SIFT-50M dataset:
{'audio_path': {'bytes': b'OggS\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00k:\xcd\x11\x00\x00\x00\x00\xc2t{$\x01\x13OpusHead\x01\x018\x01\x80>\x00\x00\x00\x00\x00OggS\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00k:\xcd\x11\x01\x00\x00\x00\xa8\xf5|\x14\x04\xff\xff\xff\xfeOpusTags\r\x00\x00\x00libopus 1.1.2\x07\x00\x00\x00&\x00\x00\x00ENCODER=opusenc from opus-tools 0.1.10\x17\x00\x00\x00ENCODER_OPTIONS=--quiet \x00\x00\x00artist=Alois Theodor SonnleitnerW\x00\x00\x00DESCRIPTION=http://www.archive.org/details/die_hoehlenkinder_im_steinhaus_1101_librivox\x1d\x00\x00\x00title=20 - Sehnsuchtsmaerchen&\x00\x00\x00album=Die Hoehlenkinder - Im Steinhaus\x15\x00\x00\x00encoder=Lavf57.83.100\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [ ]:
?os.path.splitext